In [1]:
import sys
sys.path.append("..")
from pathlib import Path
import numpy as np
from scipy import stats
import importlib
import src.assay_calibration.fit_utils.two_sample.fit
from src.assay_calibration.fit_utils.fit import Fit
importlib.reload(src.assay_calibration.fit_utils.two_sample.fit)
importlib.reload(src.assay_calibration.fit_utils.fit)
from src.assay_calibration.fit_utils.two_sample.fit import single_fit
from src.assay_calibration.fit_utils.two_sample import (density_utils,constraints, optimize)
import scipy.stats as sps
import matplotlib
matplotlib.set_loglevel("warning")
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import trange
import os
sys.path.append(str(Path(os.getcwd()).parent))
from src.assay_calibration.data_utils.dataset import (
    PillarProjectDataframe,
    Scoreset,
    BasicScoreset,
)
import json
import glob

In [2]:
# from src.assay_calibration.data_utils.dataset import (
#     PillarProjectDataframe,
#     Scoreset,
#     BasicScoreset,
# )
# ds = Scoreset.from_json("/data/ross/assay_calibration/scoresets/CALM1_CALM2_CALM3_Weile_2017.json")

In [3]:
# dir(ds)

In [4]:
# ds.scoreset_name

In [5]:
# for sample in ds.samples:
#     print(sample[1])
#     # break

In [6]:
def test_fit(ds, component_range, check_monotonic, init_strategy, bootstrap_seed=None):
    # Load directly from JSON
    fit = Fit(ds)
    fits, best_idx, best_val_ll = fit.run(
                core_limit=30,
                num_fits=100,
                verbose_level=20,
                component_range=component_range,
                bootstrap=True,
                bootstrap_seed=bootstrap_seed,
                verbose=False,
                max_em_iters=10000,
                check_convergence=False,
                check_monotonic=check_monotonic,
                # submerge_steps=None,#256 if check_monotonic else None,
                init_strategy=init_strategy,
                score_min=ds.scores.min() - 1,
                score_max=ds.scores.max() + 1,
                # init_constraint_adjustment_param="random",
                # kmean_init="k-means++"
            )
    
    return fits, ds, best_idx, best_val_ll

# for dataset_f in glob.glob("/data/ross/assay_calibration/scoresets/*.json"):
    
#     dataset_name = dataset_f.split('/')[-1][:-5]

#     for component_range in ("2-component","3-component"):
#         for monotonicity_contraint in ("no constraint","constraint"):
#             # for init_strategy in ("kmeans","random"):
#             init_strategy = "kmeans"
            
#             fits, ds = test_fit(dataset_f, component_range=[3] if component_range[0] == "3" else [2], 
#                                 check_monotonic=False if monotonicity_contraint[0] == "n" else True, 
#                                 init_strategy=init_strategy)
            
#             fit_results = sorted(fits, key=lambda res: res['likelihoods'][-1], reverse=True)
#             best_fit = fit_results[0]
#             scores = ds.scores
#             sample_assignments = ds.sample_assignments
            
#             score_range = np.linspace(scores.min(), scores.max(), 1000)
#             estimatedDensities = np.array([density_utils.joint_densities(score_range[...,None],
#                                                                          best_fit['component_params'],
#                                                                          sample_weights).squeeze() for sample_weights in best_fit['weights']])
            
#             fig,ax = plt.subplots(3,1, figsize=(12,12))
#             for i in range(3):
#                 ax[i].plot(score_range, estimatedDensities[i].sum(0), label='Estimated', color='C1', linestyle='--')
#                 ax[i].hist(scores[sample_assignments[:,i]], density=True, alpha=0.3, color='gray', label='Data histogram')
#                 ax[i].set_title(f'Sample {i+1}')
#                 ax[i].legend()

In [7]:
all_results = {}

In [8]:
#### import matplotlib.pyplot as plt
import glob
import os
import numpy as np
import pickle

# Create output directory if it doesn't exist
output_dir = "/data/ross/assay_calibration/bootstrap"
os.makedirs(output_dir, exist_ok=True)

N_BOOTSTRAPS = 3000

# Suppress matplotlib debug messages
import logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)

n_non_violating_fits_without_constraint = 0

for dataset_f in (
    glob.glob("/data/ross/assay_calibration/scoresets/BRCA1*Findlay*.json")
):
    
    dataset_name = dataset_f.split('/')[-1][:-5]
    print(f"Processing {dataset_name}")
    json_save_dir = f'{output_dir}/{dataset_name}'
    os.makedirs(json_save_dir,exist_ok=True)

    ds = Scoreset.from_json(dataset_f)
    print(ds)

    sample_names = np.array([sample[1] for sample in ds.samples])
    
    # Create figure with 4x3 subplot grid
    # fig, axes = plt.subplots(len(sample_names), 4, figsize=(18, 5*len(sample_names)))
    # fig.suptitle(f'Dataset: {dataset_name}', fontsize=16, y=0.995)

    all_results[dataset_name] = {}
    
    # plot_idx = 0
    for component_range in ["3-component"]:#["2-component", "3-component"]:
        for monotonicity_constraint in ["constraint"]:#["constraint", "no constraint"]:
            # for init_density_fix in ["lambda","sigma"]:
            init_strategy = "random"

            # for N_BOOTSTRAPS in [100,1000,5000]:

            all_results[dataset_name][(component_range, monotonicity_constraint)] = []
    
            for bootstrap_iter in range(N_BOOTSTRAPS):
                print(f'{component_range} {monotonicity_constraint} bootstrap {bootstrap_iter}/{N_BOOTSTRAPS} start...',end='')
                
                try:
                    # Run the fit
                    fits, ds, best_idx, best_val_ll = test_fit(
                        ds, 
                        component_range=[3] if component_range[0] == "3" else [2], 
                        check_monotonic=False if monotonicity_constraint[0] == "n" else True, 
                        init_strategy=init_strategy,
                        bootstrap_seed=bootstrap_iter
                    )

                    print(f' val_ll: {best_val_ll}')
                    
                    # Get best fit
                    fit_results = sorted(fits, key=lambda res: res['likelihoods'][-1], reverse=True)
                    times_submerged = [results['times_submerged'] for results in fit_results if len(results['times_submerged']) != 0]
                    print(dataset_name, component_range, monotonicity_constraint, times_submerged)
                    # best_fit = fit_results[0]
                    best_fit = fit_results[best_idx]
                    best_init = "MoM" if best_fit['kmeans'] == "method_of_moments" else "KM"
                    xlims = best_fit['xlims']
                    scores = ds.scores
                    sample_assignments = ds.sample_assignments
        

                    save_dict = {'fit': best_fit, 
                                 'times_submerged': times_submerged,
                                 'best_val_ll': best_val_ll}
                    all_results[dataset_name][(component_range, monotonicity_constraint)].append(save_dict)

                    with open(f'{json_save_dir}/3c_con_{bootstrap_iter}.pkl', 'wb') as f:
                        pickle.dump(save_dict, f)
                    
        
                    # Calculate densities
                    # score_range = np.linspace(scores.min(), scores.max(), 1000)
                    # estimatedDensities = np.array([
                    #     density_utils.joint_densities(
                    #         score_range[..., None],
                    #         best_fit['component_params'],
                    #         sample_weights
                    #     ).squeeze() for sample_weights in best_fit['weights']
                    # ])
                    
                    # # Check if density constraint violated
                    # fit_violates_constraint = constraints.multicomponent_density_constraint_violated(best_fit['component_params'], xlims)
                    # if monotonicity_constraint == "no constraint" and fit_violates_constraint:
                    #     n_non_violating_fits_without_constraint += 1
                    # fit_violates_constraint = "violates" if fit_violates_constraint else "not violates"
                    
                    # # Plot for each sample (3 columns)
                    # for i in range(len(estimatedDensities)):
                    #     ax = axes[i, plot_idx]
                        
                    #     # Plot estimated density
                    #     ax.plot(score_range, estimatedDensities[i].sum(0), 
                    #            label='Estimated', color='C1', linestyle='-', linewidth=2)
                        
                    #     # Plot histogram of actual data
                    #     if i < sample_assignments.shape[1] and sample_assignments[:, i].sum() > 0:
                    #         ax.hist(scores[sample_assignments[:, i]], 
                    #                bins=30, density=True, alpha=0.3, 
                    #                color='gray', label='Data')
                        
                    #     # Set labels and title
                    #     if plot_idx == 0:
                    #         ax.set_ylabel(sample_names[i], 
                    #                      fontsize=12, fontweight='bold')
                        
                    #     if i == 0:
                    #         ax.set_title(f'{component_range}, {monotonicity_constraint}, b:{N_BOOTSTRAPS}, {best_init}, {fit_violates_constraint}', fontsize=12)
                        
                    #     if i == len(estimatedDensities) - 1:
                    #         ax.set_xlabel('Score', fontsize=10)
                        
                    #     ax.legend(loc='upper right', fontsize=8)
                    #     ax.grid(True, alpha=0.2)
                        
                    #     # Add likelihood value as text
                    #     likelihood = best_fit['likelihoods'][-1]
                    #     ax.text(0.02, 0.98, f'LL: {likelihood:.3f}, n={len(scores[sample_assignments[:, i]])}', 
                    #            transform=ax.transAxes, fontsize=12,
                    #            verticalalignment='top')
                    
                except Exception as e:
                    print(f"  Error with {component_range}, {monotonicity_constraint} bootstrap {bootstrap_iter}/{N_BOOTSTRAPS}: {e}")
                    # Create empty plots with error message
                    # for i in range(len(estimatedDensities)):
                    #     ax = axes[i, plot_idx]
                    #     ax.text(0.5, 0.5, f'Error:\n{str(e)[:30]}...', 
                    #            ha='center', va='center', transform=ax.transAxes)
                    #     ax.set_xticks([])
                    #     ax.set_yticks([])
    
                    #     if i == 0:
                    #         ax.set_title(f'{component_range}, {monotonicity_constraint}, b:{N_BOOTSTRAPS}, {best_init}, {fit_violates_constraint}', fontsize=12)
                        
                    #     if plot_idx == 0:
                    #         ax.set_ylabel(sample_names[i], 
                    #                      fontsize=12, fontweight='bold')
                        
                    #     if i == len(estimatedDensities) - 1:
                    #         ax.set_xlabel('Score', fontsize=10)
            
            # plot_idx += 1
    
    # Adjust layout and save
    # plt.tight_layout(rect=[0, 0, 1, 0.99])
    
    # output_path = f"{output_dir}/{dataset_name}.png"
    # plt.savefig(output_path, dpi=100, bbox_inches='tight')
    # plt.close()
    
    # print(f"  Saved to {output_path}")

# print("All plots saved!")

Processing BRCA1_Findlay_2018
BRCA1_Findlay_2018: 3893 total variants
	Pathogenic/Likely Pathogenic: 410 variants
	Benign/Likely Benign: 270 variants
	gnomAD: 599 variants
	Synonymous: 544 variants

3-component constraint bootstrap 0/3000 start... val_ll: -0.5771698730674224
BRCA1_Findlay_2018 3-component constraint []
3-component constraint bootstrap 1/3000 start... val_ll: -0.6057737766813108
BRCA1_Findlay_2018 3-component constraint []
3-component constraint bootstrap 2/3000 start... val_ll: -0.5657873312132117
BRCA1_Findlay_2018 3-component constraint []
3-component constraint bootstrap 3/3000 start... val_ll: -0.583081740908234
BRCA1_Findlay_2018 3-component constraint []
3-component constraint bootstrap 4/3000 start... val_ll: -0.65177591614315
BRCA1_Findlay_2018 3-component constraint []
3-component constraint bootstrap 5/3000 start... val_ll: -0.6298001841199572
BRCA1_Findlay_2018 3-component constraint []
3-component constraint bootstrap 6/3000 start... val_ll: -0.648080150175

/home/rcstewart/assay_calibration/src/assay_calibration/fit_utils/two_sample/density_utils.py:110: RuntimeWarning: divide by zero encountered in log
  Likelihood += np.log(sample_likelihood).sum().item()


 val_ll: -0.7166213522488044
BRCA1_Findlay_2018 3-component constraint []
3-component constraint bootstrap 38/3000 start... val_ll: -0.6197657428121612
BRCA1_Findlay_2018 3-component constraint []
3-component constraint bootstrap 39/3000 start... val_ll: -0.6178955910754017
BRCA1_Findlay_2018 3-component constraint []
3-component constraint bootstrap 40/3000 start... val_ll: -0.6111650705851708
BRCA1_Findlay_2018 3-component constraint []
3-component constraint bootstrap 41/3000 start... val_ll: -0.6328633400846094
BRCA1_Findlay_2018 3-component constraint []
3-component constraint bootstrap 42/3000 start... val_ll: -0.656221801557413
BRCA1_Findlay_2018 3-component constraint []
3-component constraint bootstrap 43/3000 start... val_ll: -0.6246521448498443
BRCA1_Findlay_2018 3-component constraint []
3-component constraint bootstrap 44/3000 start... val_ll: -0.6136270099762714
BRCA1_Findlay_2018 3-component constraint []
3-component constraint bootstrap 45/3000 start... val_ll: -0.63552

In [ ]:
val_lls = {'3-component':[], '2-component':[]} # 100 same bootstraps
for dataset in all_results:
    for key in all_results[dataset]:
        for bootstrap_results in all_results[dataset][key]:
            val_lls[key[0]].append(bootstrap_results['best_val_ll'])


In [12]:
# for dataset_f in glob.glob("/data/ross/assay_calibration/scoresets/KCNH2_O_Neill_2024_surface_expression*.json"):
#     dataset_name = dataset_f.split('/')[-1][:-5]
#     print(f"Processing {dataset_name}")

#     ds = Scoreset.from_json(dataset_f)
#     print(ds)

#     sample_names = np.array([sample[1] for sample in ds.samples])
    
#     # Create figure with 4x3 subplot grid
#     fig, axes = plt.subplots(len(sample_names), 4, figsize=(18, 5*len(sample_names)))
#     fig.suptitle(f'Dataset: {dataset_name}', fontsize=16, y=0.995)
    
#     for i,component_range in enumerate(["2-component", "3-component"]):
#         for monotonicity_constraint in ["constraint", "no constraint"]:
#             if (component_range,monotonicity_constraint) not in all_results[dataset_name]:
#                 continue
#             best_fit = all_results[dataset_name][(component_range,monotonicity_constraint)]
        
#             best_init = "MoM" if best_fit['kmeans'] == "method_of_moments" else "KM"
#             xlims = best_fit['xlims']
#             scores = ds.scores
#             sample_assignments = ds.sample_assignments


#             # Calculate densities
#             score_range = np.linspace(scores.min(), scores.max(), 1000)
#             estimatedDensities = np.array([
#                 density_utils.joint_densities(
#                     score_range[..., None],
#                     best_fit['component_params'],
#                     sample_weights
#                 ).squeeze() for sample_weights in best_fit['weights']
#             ])
            
#             # Check if density constraint violated
#             fit_violates_constraint = constraints.multicomponent_density_constraint_violated(best_fit['component_params'], xlims)
#             if monotonicity_constraint == "no constraint" and fit_violates_constraint:
#                 n_non_violating_fits_without_constraint += 1
#             fit_violates_constraint = "violates" if fit_violates_constraint else "not violates"
            
#             # Plot for each sample (3 columns)
#             for i in range(len(estimatedDensities)):
#                 ax = axes[i, plot_idx]
                
#                 # Plot estimated density
#                 ax.plot(score_range, estimatedDensities[i].sum(0), 
#                        label='Estimated', color='C1', linestyle='-', linewidth=2)
                
#                 # Plot histogram of actual data
#                 if i < sample_assignments.shape[1] and sample_assignments[:, i].sum() > 0:
#                     ax.hist(scores[sample_assignments[:, i]], 
#                            bins=30, density=True, alpha=0.3, 
#                            color='gray', label='Data')
                
#                 # Set labels and title
#                 if plot_idx == 0:
#                     ax.set_ylabel(sample_names[i], 
#                                  fontsize=12, fontweight='bold')
                
#                 if i == 0:
#                     ax.set_title(f'{component_range}, {monotonicity_constraint}, {best_init}, {fit_violates_constraint}', fontsize=12)
                
#                 if i == len(estimatedDensities) - 1:
#                     ax.set_xlabel('Score', fontsize=10)
                
#                 ax.legend(loc='upper right', fontsize=8)
#                 ax.grid(True, alpha=0.2)
                
#                 # Add likelihood value as text
#                 likelihood = best_fit['likelihoods'][-1]
#                 ax.text(0.02, 0.98, f'LL: {likelihood:.3f}, n={len(scores[sample_assignments[:, i]])}', 
#                        transform=ax.transAxes, fontsize=12,
#                        verticalalignment='top')
            
        
#         plot_idx += 1

# # Adjust layout and save
# plt.tight_layout(rect=[0, 0, 1, 0.99])


In [21]:
n_non_violating_fits_without_constraint = [0,0]
n_violating_fits_without_constraint = [0,0]
for dataset_f in glob.glob("/data/ross/assay_calibration/scoresets/*.json"):
    dataset_name = dataset_f.split('/')[-1][:-5]
    for i,component_range in enumerate(["2-component", "3-component"]):
        for monotonicity_constraint in ["constraint", "no constraint"]:
            if (component_range,monotonicity_constraint) not in all_results[dataset_name]:
                continue
            params = all_results[dataset_name][(component_range,monotonicity_constraint)]['component_params']
            xlims = all_results[dataset_name][(component_range,monotonicity_constraint)]['xlims']
            fit_violates_constraint = constraints.multicomponent_density_constraint_violated(params, xlims)
            if monotonicity_constraint == "no constraint" and fit_violates_constraint:
                n_violating_fits_without_constraint[i] += 1
            elif monotonicity_constraint == "no constraint":
                n_non_violating_fits_without_constraint[i] += 1

for i,component_range in enumerate(["2-component", "3-component"]):
    print(f'{component_range}: {100*n_non_violating_fits_without_constraint[i]/(n_non_violating_fits_without_constraint[i]+n_violating_fits_without_constraint[i]):.1f}% ({n_non_violating_fits_without_constraint[i]}/{n_non_violating_fits_without_constraint[i]+n_violating_fits_without_constraint[i]}) did not violate the constraint.')

                

2-component: 44.4% (36/81) did not violate the constraint.
3-component: 12.8% (10/78) did not violate the constraint.


In [12]:
temp = 0

temp

166

In [10]:
ds = Scoreset.from_json('/data/ross/assay_calibration/scoresets/CALM1_CALM2_CALM3_Weile_2017.json')
ds.scores[ds.sample_assignments[:,2]]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
# 'HMBS_van_Loggerenberg_2023_combined', 'TP53_Fortuno_2021_Kato_meta' truth value of arr
# 'KCNH2_Jiang_2022', 'KCNH2_O_Neill_2024_surface_expression' ll decreased consistently & xlims



In [8]:
constraints.multicomponent_density_constraint_violated(best_fit['component_params'],
                                                       score_range[[0,-1]])

True

In [9]:
best_fit.keys()

dict_keys(['component_params', 'weights', 'likelihoods', 'history', 'kmeans'])

In [10]:
best_fit['component_params']

[(-8.553172420104763, 0.7949678509378952, 0.24375170761527565),
 (-1.4489019235176928, 0.9296749533938731, 0.04216918377139136),
 (-7.860341896888932e-05, 1.003620376681616, 0.020770124957920067)]

In [11]:
scores.min(),scores.max()

(np.float64(-0.483185729440366), np.float64(1.36169269063829))

In [12]:
# Iteration 9618: Likelihood DECREASED by 1.96e-06 (relative: 3.51e-06) - algorithmic issue